

# CSE 4693/6693 Intro to Machine Learning

## Project: LSTM-GAN Based Wireless Channel Modeling

<div style="float: left; margin-right: 20px;">

| Name | NetID |
|:-----|:------|
| Joshua Moore | jjm702 |
| Tirian Judy | tkj105 |
| Claire Johnson | kj1289 |
| Aayam Raj Shakya | as5160 |

</div>

In [2]:
# Imports & some predefines
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import os
import glob
import pandas as pd
import pickle

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Use cuda if possible; otherwise eat some threads on the CPU

In [3]:
# Data parsing

'''
This is the data we want to load
Distance (dist): Between the UAV and the receiver.
Altitude (alt): UAV height above ground.
Frequency (freq): Operating frequencies measured.
UAV Speed (vel_x, vel_y): UAV mobility, affecting Doppler shifts.
Environment Type: Use categorical encoding for rural/open field or terrain information.
LOS/NLOS Classification: A binary or probabilistic input indicating line-of-sight status.
'''

CENTER_FREQ = 3564
file_path = 'dataset/'

# Read all CSV files
arrays = [pd.read_csv(file).rename(columns=lambda x: x.strip()) for file in glob.glob(file_path + "*.csv")]

# Filter the columns in each dataframe, and include only the ones that exist in the dataframe
filtered_arrays = []

'''
path loss ewma is ...
aod_theta, aod_phi: Angles of Departure in the vertical (elevation) and horizontal (azimuth) planes
aoa_theta, aoa_phi: Angles of Arrival in the vertical (elevation) and horizontal (azimuth) planes
These angles are critical for beamforming, MIMO systems, and propagation modeling, as they help in
determining the direction of signal transmission (AOD) and reception (AOA), improving channel
estimation and system optimization.

AOD (Angle of Departure) and AOA (Angle of Arrival) are calculated based on the relative positions
of the transmitter (UAV) and receiver (target) in both horizontal (azimuth) and vertical (elevation) planes.

Horizontal (Azimuth) Angle (phi):
phi = atan2((y_target - y_uav), (x_target - x_uav))

Vertical (Elevation) Angle (theta):
theta = atan2((z_target - z_uav), sqrt((x_target - x_uav)^2 + (y_target - y_uav)^2))

These angles are essential for beamforming, signal propagation modeling, and estimating the channel conditions.

for original peaks and peaks we need to process data and only include when it has 4 items in the array
'peaks' represents the indices where the processed signal exhibits local maxima, potentially indicating key events such as
moments of high signal strength or important changes in the UAV's movement (e.g., rapid shifts in altitude or speed).
'orig_peaks' shows the indices of maxima in the raw, unprocessed data, which could reflect more noise or less clarity.
These peak points are essential for analyzing signal behavior, especially in the context of parameters like distance ('dist'),
altitude ('alt'), frequency offset should change, and UAV velocity ('vel_x', 'vel_y') this tells us if the UAV is moving, as they might correlate with specific moments
when the signal is strongest or when significant changes in the UAV’s position or mobility occur.
'''

columns_to_keep = ['dist', 'avgSnr', 'freq_offset','avgPower', 'avg_pl','avg_pl_ewma', 'aod_theta','aoa_theta', 'peaks', 'orig_peaks','speed', 'stage', 'vel_x', 'vel_y','vel_z']

# Process the data
for df in arrays:
    # Keep only the relevant columns that exist in the dataframe
    existing_columns = [col for col in columns_to_keep if col in df.columns.str.strip()]
    filtered_df = df[existing_columns]

    if 'stage' in filtered_df.columns:
        filtered_df = filtered_df[filtered_df['stage'] == 'Flight']

    # if 'peaks' in filtered_df.columns:
    #     filtered_df['peaks'] = filtered_df['peaks'].apply(lambda x: [i for i in x.split() if i.strip()][1:-1] if isinstance(x, str) else x)
    #     print(filtered_df['peaks'])

    filtered_arrays.append(filtered_df)

# Print the first 5 rows of the first dataframe (for verification)
# print(filtered_arrays[0].head())

In [7]:
# Build the Generator
class Generator(nn.Module):

    def __init__(self, inputSize=6, outputSize =(48,48)): # LSTM model should produce a vector of 6 terms; Output an array of 48H * 48W
        super(Generator, self).__init__() # We should be fine using Pytorch base model code

        self.outputSize = outputSize # Need to remember the output size for the forward function

        self.model = nn.Sequential(nn.Linear(inputSize, 128), # Mapping input vector to 128 neurons;
                                   nn.LeakyReLU(0.2, inplace=True), # Using inplace=True to save memory + it seems to be standard practice
                                   # Start hidden layers
                                   nn.Linear(128,256), # Hidden Layer 1; Upscaling by factor of 2
                                   nn.BatchNorm1d(256), # Normalization
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(256, 512), # Hidden Layer 2; Upsacling by factor of 2; Last hidden layer for now
                                   nn.BatchNorm1d(512), # Normalization
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(512, outputSize[0] * outputSize[1]), # Output layer
                                   nn.Tanh() # Normalization for output (-1, 1)
                                   )

    # Passes data into first layer & executes sequentially based upon params from self.model()
    def forward(self, noise):
        genImage =  self.model(noise) # Grab output tensor
        return genImage.view(-1, 1, self.outputSize[0], self.outputSize[1]) # Should produce a grey scale image using the view function. Args: Batchsize(-1 means figure it out for me), numChannels, height, width

In [8]:
# Build the Discriminator
class Discriminator(nn.Module):

    def __init__(self, inputSize=(48,48)): # Simple classifier returns [0,1]; 1 real
        super(Discriminator, self).__init__() # Use the base Pytorch discriminator

        self.inputSize = inputSize # Need to remember the input size for the forward function

        self.model = nn.Sequential(nn.Linear(inputSize[0] * inputSize[1], 512), # Mapping input vector to 512 neurons
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(512, 256), # Hidden Layer 1; Downscaling by factor of 2
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(256, 1), # Output layer - Map 256 neurons into the probability of being real
                                   nn.Sigmoid() # Turn the output into [0,1]
                                   )

    def forward(self, input):
        flatInput = input.view(input.size(0), -1) # Flatten the input to 1D
        return self.model(flatInput)

In [10]:
# Initializing GAN Components

# Loss function
losFunc = nn.BCELoss()

# Creating the Discriminator
discrim = Discriminator().to(DEVICE)
dOptimizer = optim.Adam(discrim.parameters(), lr=0.0002, betas=(0.5, 0.999)) # Lower B1 since models will fight each other

# Creating the Generator
generator = Generator().to(DEVICE)
gOptimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999)) # Lower B1 since models will fight each other